In [1]:
# Imports
from llmsql.config import Arguments, SettingsManager, Settings
from llmsql.llm import ModelManager
from llmsql.db import Database
from dotenv import load_dotenv

c:\Users\FADELCO\miniconda3\envs\demo-nigelec\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib.util>:262: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.


In [2]:
db = Database(db_name="dvdrental",user_name="postgres",pwd="841148",host="localhost",port=5432)

In [3]:
sql_database = db.get_sql_db()

Successfully connected to the PostgreSQL database


In [4]:
dir(sql_database)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_all_tables',
 '_custom_table_info',
 '_engine',
 '_ignore_tables',
 '_include_tables',
 '_indexes_in_table_info',
 '_inspector',
 '_max_string_length',
 '_metadata',
 '_sample_rows_in_table_info',
 '_schema',
 '_usable_tables',
 'dialect',
 'engine',
 'from_uri',
 'get_single_table_info',
 'get_table_columns',
 'get_usable_table_names',
 'insert_into_table',
 'metadata_obj',
 'run_sql',
 'truncate_word']

In [5]:
tables = sql_database.get_usable_table_names()
tables

['actor',
 'address',
 'category',
 'city',
 'country',
 'customer',
 'film',
 'film_actor',
 'film_category',
 'inventory',
 'language',
 'payment',
 'rental',
 'staff',
 'store']

In [7]:
# load env variables
load_dotenv("../.env")

# Set llm
args = Arguments()

args.llm_type = 'gemini'
args.llm_name = 'models/gemini-1.5-flash'
args.temperature = 0.6
args.tokenizer_name = 'Qwen/Qwen2.5-7B-Instruct'

args.context_window = 512
args.num_output = 2048

args.is_openvino_embed = True
args.embed_model = r"D:\llm-sql\models\kalm_ov"

SettingsManager.set_settings(args=args)

In [8]:
# sanity check
for a in (Settings.llm,Settings.tokenizer,Settings.num_output,Settings.context_window):
    print(a,end="\n\n")

callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x00000254BE79DB10> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x00000254A66D4CC0> completion_to_prompt=<function default_completion_to_prompt at 0x00000254A679AA20> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None model='models/gemini-1.5-flash' temperature=0.3 max_tokens=256 generate_kwargs={'system_prompt': None}

functools.partial(<bound method PreTrainedTokenizerBase.encode of Qwen2TokenizerFast(name_or_path='Qwen/Qwen2.5-7B-Instruct', vocab_size=151643, model_max_length=8000, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|visi

In [9]:
db.build_query_engine(sql_database=sql_database,
                      tables=sql_database.get_usable_table_names(),
                      sql_table_schemas=None)

query engine has been built.


In [13]:
response = db.query_text("How many  unique actors are there?")

In [16]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='b0ad2a0d-210d-4ae5-9369-8aced9f35ac4', embedding=None, metadata={'sql_query': 'SELECT COUNT(DISTINCT actor_id) FROM actor;', 'result': [(200,)], 'col_keys': ['count']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='[(200,)]', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)]

In [15]:
response.metadata['sql_query']

'SELECT COUNT(DISTINCT actor_id) FROM actor;'